In [ ]:
!pip install evaluate
!pip install bert_score
!pip install datasets
!pip install -U sentence-transformers

In [ ]:
import tensorflow as tf
import numpy as np
import sklearn
import transformers
import json
import matplotlib.pyplot as plt
import random
import seaborn as sn
from sklearn import metrics

from datasets import load_dataset
from evaluate import load

In [ ]:
dataset = load_dataset("banking77")

In [ ]:
bertscore = load("bertscore")

In [ ]:
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")
results

{'precision': [0.9999999403953552, 1.0],
 'recall': [0.9999999403953552, 1.0],
 'f1': [0.9999999403953552, 1.0],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.28.1)'}

In [ ]:
results['precision'][0]

0.9999999403953552

In [ ]:
# Берём тексты и лейблы для трейн части датасета
texts = dataset['train']['text']
labels = dataset['train']['label']
x = zip(labels, texts)
xs = sorted(x, key=lambda a: a[0])
sorted_labels = [x[0] for x in xs]
sorted_texts = [x[1] for x in xs]
# Проверяем что правильно отсортировали
for i in range(len(sorted_labels)):
  if sorted_labels[i] != labels[texts.index(sorted_texts[i])]:
    print('err')
labels = sorted_labels
texts = sorted_texts

train = []
for i in range(77):
  train.append([])
for i in range(len(texts)):
  label = labels[i]
  train[label].append(texts[i])

In [ ]:
from google.colab import drive
import pandas as pd 

drive.mount('/content/drive')
path = '/content/drive/My Drive/nlp_aug/norm_aug.csv'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv(path)

In [ ]:
df.at[2, 'text']

'My new card needs to be activated before I can use it.'

In [ ]:
df['bert-score'] = 0

In [ ]:
df

,Unnamed: 0,class,text,bert-score
0,0,0,I can't seem to get my card activated.,0
1,1,0,Help me with the process of activating my card...,0
2,2,0,My new card needs to be activated before I can...,0
3,3,0,Can you assist me in activating my card?,0
4,4,0,What steps do I need to follow to activate my ...,0
...,...,...,...,...
9859,9859,76,It's frustrating when the ATM doesn't accurate...,0
9860,9860,76,I prefer using a credit card for international...,0
9861,9861,76,I made sure to double-check the current exchan...,0
9862,9862,76,There is always some uncertainty surrounding i...,0


In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
def check_aug_sentence_by_all_class_texts(sentence: str, label: int):
  sentences = [train[label], [sentence] * len(train[0])]

  embedding_1= model.encode(sentences[0], convert_to_tensor=True)
  embedding_2 = model.encode(sentences[1], convert_to_tensor=True)
  res = util.pytorch_cos_sim(embedding_1, embedding_2)

  answer = 0
  for r in res:
      answer += r[0].item()
  answer /= len(res)
  return answer

In [ ]:
for i in range(len(df)):
  sentence = df.at[i, 'text']
  label = int(df.at[i, 'class'])
  result = check_aug_sentence_by_all_class_texts(sentence, label)
  print(i, label, result)
  df.at[i, 'bert-score'] = result